In [1]:
import torch

In [2]:
torch.cuda.is_available()

True

In [3]:
import torch_scatter

In [4]:
import argparse
import random
import os
import pickle
import glob
import json
import time
import logging
import re
from itertools import chain
from string import punctuation

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

import torch
from torch import nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl

import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm
# from tqdm.notebook import tqdm_notebook as tqdm

from transformers import AdamW, get_linear_schedule_with_warmup

[nltk_data] Downloading package punkt to /home/yw4509/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
from table_bert import TableBertModel
from table_bert import Table, Column

### Data Processing to Combine Tables with files

In [6]:
pd.set_option('display.max_colwidth', -1)

In [4]:
test_f = pd.read_json("./data/test_files.json")
train_f = pd.read_json("./data/train_files.json")
dev_f = pd.read_json("./data/dev_files.json")
print(test_f.columns)
test_f.head()

Index(['phase', 'table_id', 'question', 'sql', 'answer', 'sql_query'], dtype='object')


,phase,table_id,question,sql,answer,sql_query
0,1,1-10015132-16,What is terrence ross' nationality,"{'sel': 2, 'conds': [[0, 0, 'Terrence Ross']], 'agg': 0}",[united states],"{'agg_index': 0, 'cond_ops': ['=', '>', '<', 'OP'], 'ordered': True, 'syms': ['SELECT', 'WHERE', 'AND', 'COL', 'TABLE', 'CAPTION', 'PAGE', 'SECTION', 'OP', 'COND', 'QUESTION', 'AGG', 'AGGOPS', 'CONDOPS']}"
1,1,1-10015132-16,What clu was in toronto 1995-96,"{'sel': 5, 'conds': [[4, 0, '1995-96']], 'agg': 0}",[arkansas],"{'agg_index': 0, 'cond_ops': ['=', '>', '<', 'OP'], 'ordered': True, 'syms': ['SELECT', 'WHERE', 'AND', 'COL', 'TABLE', 'CAPTION', 'PAGE', 'SECTION', 'OP', 'COND', 'QUESTION', 'AGG', 'AGGOPS', 'CONDOPS']}"
2,1,1-10015132-16,which club was in toronto 2003-06,"{'sel': 5, 'conds': [[4, 0, '2003-06']], 'agg': 0}",[michigan],"{'agg_index': 0, 'cond_ops': ['=', '>', '<', 'OP'], 'ordered': True, 'syms': ['SELECT', 'WHERE', 'AND', 'COL', 'TABLE', 'CAPTION', 'PAGE', 'SECTION', 'OP', 'COND', 'QUESTION', 'AGG', 'AGGOPS', 'CONDOPS']}"
3,1,1-10015132-16,how many schools or teams had jalen rose,"{'sel': 5, 'conds': [[0, 0, 'Jalen Rose']], 'agg': 3}",[1],"{'agg_index': 3, 'cond_ops': ['=', '>', '<', 'OP'], 'ordered': True, 'syms': ['SELECT', 'WHERE', 'AND', 'COL', 'TABLE', 'CAPTION', 'PAGE', 'SECTION', 'OP', 'COND', 'QUESTION', 'AGG', 'AGGOPS', 'CONDOPS']}"
4,1,1-10083598-1,Where was Assen held?,"{'sel': 2, 'conds': [[3, 0, 'Assen']], 'agg': 0}",[netherlands],"{'agg_index': 0, 'cond_ops': ['=', '>', '<', 'OP'], 'ordered': True, 'syms': ['SELECT', 'WHERE', 'AND', 'COL', 'TABLE', 'CAPTION', 'PAGE', 'SECTION', 'OP', 'COND', 'QUESTION', 'AGG', 'AGGOPS', 'CONDOPS']}"


In [3]:
test_tb = spark.read.json("./data/test_tables.jsonl").toPandas().rename(columns={'id':'table_id'})
train_tb = spark.read.json("./data/train_tables.jsonl").toPandas().rename(columns={'id':'table_id'})
dev_tb = spark.read.json("./data/dev_tables.jsonl").toPandas().rename(columns={'id':'table_id'})
test_tb.loc[0]

caption                                                          R
header           [Player, No., Nationality, Position, Years in ...
table_id                                             1-10015132-16
name                                             table_10015132_16
page_id                                                        NaN
page_title                         Toronto Raptors all-time roster
rows             [[Aleksandar Radojević, 25, Serbia, Center, 19...
section_title                                                    R
types                         [text, text, text, text, text, text]
Name: 0, dtype: object

In [4]:
def header_trans(df):
    header_list = []
    for i, row in df.iterrows():
        header_new = list(zip(row['header'], row['types'],row['rows'][0]))
        header_list.append(header_new) 
    df['header_new']=header_list
    df = df.drop(columns=['header'])
    return df

In [5]:
test_tb= header_trans(test_tb)
train_tb= header_trans(train_tb)
dev_tb= header_trans(dev_tb)

In [6]:
test_tb.loc[0]

caption                                                          R
table_id                                             1-10015132-16
name                                             table_10015132_16
page_id                                                        NaN
page_title                         Toronto Raptors all-time roster
rows             [[Aleksandar Radojević, 25, Serbia, Center, 19...
section_title                                                    R
types                         [text, text, text, text, text, text]
header_new       [(Player, text, Aleksandar Radojević), (No., t...
Name: 0, dtype: object

In [7]:
test = pd.merge(test_f,test_tb,on=['table_id']).rename(columns={'question':'context','page_title':'title','header_new':'header'})
train = pd.merge(train_f,train_tb,on=['table_id']).rename(columns={'question':'context','page_title':'title','header_new':'header'})
dev = pd.merge(dev_f,dev_tb,on=['table_id']).rename(columns={'question':'context','page_title':'title','header_new':'header'})

In [10]:
test.iloc[1]

phase            1                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       
table_id         1-10015132-16                                                                                                                                                                                                                                                                                                                                                                                                                

### Remove Empty Ans from the Database

In [13]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [31]:
def cleaning(df):
    temp=[]
    temp_len=[]
    print('original len',len(df))
    for ans in df.answer:
        inp=str(ans[0])
        ans_tok = tokenizer.tokenize(inp)
        temp.append(ans_tok)
        temp_len.append(len(ans_tok))
    df['seq_len']=temp_len
    df = df.loc[df.seq_len!=0]
    print('after cleaning',len(df))
    return df

In [32]:
train = cleaning(train)

original len 40606
after cleaning 40605


In [33]:
dev = cleaning(dev)

original len 6017
after cleaning 6017


In [34]:
test = cleaning(test)

original len 11324
after cleaning 11323


In [35]:
test.to_json('./data/test_tabert.json')
train.to_json('./data/train_tabert.json')
dev.to_json('./data/dev_tabert.json')

### subset of traning sample 100 roles

In [6]:
train = pd.read_json('./data/train_tabert.json')
dev = pd.read_json('./data/dev_tabert.json')
test = pd.read_json('./data/test_tabert.json')

In [7]:
print(len(train))
print(len(dev))
print(len(test))

40605
6017
11323


In [8]:
train_subset = train.sample(n=100)
dev_subset = dev.sample(n=100)
test_subset = test.sample(n=100)
print(len(train_subset),len(dev_subset),len(test_subset))
train_subset.to_json('./data/train_tabert_100.json')
dev_subset.to_json('./data/dev_tabert_100.json')
test_subset.to_json('./data/test_tabert_100.json')

100 100 100


In [40]:
train_subset = train.sample(frac=0.01)
dev_subset = dev.sample(frac=0.01)
test_subset = test.sample(frac=0.01)

In [41]:
print(len(train_subset),len(dev_subset),len(test_subset))

406 60 113


In [42]:
train_subset.to_json('./data/train_tabert_0.01.json')
dev_subset.to_json('./data/dev_tabert_0.01.json')
test_subset.to_json('./data/test_tabert_0.01.json')